## Data Splitting

In [1]:
from libraries.pdb import RcsbPdbClusters
import pandas as pd
from tqdm import tqdm
from src.data_preprocessing import  DataPreProcessorForGNM
from src.data_preprocessing import is_chain_valid
import biotite.structure as struc
import warnings
from libraries.lmdb_dataset import LMDBDataset
import os
from src.data_loaders import load_labels


### Enzyme Commission Number Dataset

Firstly, we need to prepare cluster fatching object.

In [2]:
clusters = RcsbPdbClusters(cluster_dir = ".\data\enzyme_data", identity=50)

2023-04-30 18:34:27.154 | INFO     | libraries.pdb:_fetch_cluster_file:80 - cluster file path: .\data\enzyme_data\pdb_clusters_50.txt


Now we want to load all the proteins and their labels.

In [3]:
labels = load_labels("./data/enzyme_data/chain_functions.txt")
labels[['pdb_id','chain']] = labels['chain_id'].str.split('.',n=1,expand=True)

Now that we have done loading, we need to get all proteins for which we do not have structure in LMDB database.

In [4]:
database = LMDBDataset("./data/pdb_data")

warnings.filterwarnings("error")
flex = "msqf"

missing_ids = []
for id in tqdm(set(labels.chain_id)):
    PDBid, chain = id.split(".")
    if(PDBid in database):
        structure = database[PDBid]["structure"]
        x = DataPreProcessorForGNM(type_flexibility = flex, num_modes=10)
        protein_chain = structure[(structure.chain_id == chain) & struc.filter_amino_acids(structure)] 
        if(not is_chain_valid(protein_chain)):
            missing_ids.extend([id])
    else:
        missing_ids.extend([id])
len(missing_ids)

100%|██████████| 37426/37426 [07:29<00:00, 83.34it/s] 


1733

Now we are only going to take sequences for which we have obtained full structures in our LMDB database.

In [5]:
labels = labels[~labels.chain_id.isin(missing_ids)]
labels

,chain_id,label,pdb_id,chain
0,4fae.B,247,4fae,B
1,3o9d.A,247,3o9d,A
2,2rkg.B,247,2rkg,B
3,3r0w.B,247,3r0w,B
4,2qmp.B,247,2qmp,B
...,...,...,...,...
37421,1z56.C,1,1z56,C
37422,4eag.A,357,4eag,A
37423,1cfr.A,5,1cfr,A
37424,2lhi.A,118,2lhi,A


Now that we have done this, it is important to attach clusters to sequences.

In [6]:
labels['clusters'] = labels.apply(lambda x: clusters.get_seqclust(pdb_code=x['pdb_id'], chain_id=x['chain']), axis=1)

2023-04-30 18:43:12.338 | WARNING  | libraries.pdb:get_seqclust:135 - Assigning cluster for obsolete entry via superceding: 5jn1->6uzu A
2023-04-30 18:47:39.087 | WARNING  | libraries.pdb:get_seqclust:135 - Assigning cluster for obsolete entry via superceding: 4iqq->5noo A
2023-04-30 18:47:39.305 | WARNING  | libraries.pdb:get_seqclust:135 - Assigning cluster for obsolete entry via superceding: 4iqq->5noo C
2023-04-30 18:49:26.740 | WARNING  | libraries.pdb:get_seqclust:135 - Assigning cluster for obsolete entry via superceding: 5urr->7mtx D
2023-04-30 18:50:04.122 | WARNING  | libraries.pdb:get_seqclust:135 - Assigning cluster for obsolete entry via superceding: 5byh->5nwt B
2023-04-30 18:51:04.746 | WARNING  | libraries.pdb:get_seqclust:135 - Assigning cluster for obsolete entry via superceding: 4wto->6kjb A
2023-04-30 18:51:25.277 | WARNING  | libraries.pdb:get_seqclust:135 - Assigning cluster for obsolete entry via superceding: 4ror->5ulv A
2023-04-30 18:51:32.623 | WARNING  | libr

Now it is important to check structure of the data that we got.

In [8]:
labels.clusters.value_counts()

108       519
33        276
41        262
266       247
63        221
         ... 
128427      1
52376       1
130437      1
133217      1
132599      1
Name: clusters, Length: 5131, dtype: int64

In [9]:
labels.label.value_counts()

357    1506
351    1062
23      836
215     694
162     626
       ... 
280      14
200      13
206       8
291       8
196       7
Name: label, Length: 384, dtype: int64

In [11]:
labels[labels.label==291]

,chain_id,label,pdb_id,chain,clusters
35964,5dud.D,291,5dud,D,25995
35965,5dud.B,291,5dud,B,25995
35966,5dud.A,291,5dud,A,32016
35967,5dud.C,291,5dud,C,32016
36603,2xu2.A,291,2xu2,A,9752
36890,2dfa.A,291,2dfa,A,627
37309,1v6t.A,291,1v6t,A,627
37407,1xw8.A,291,1xw8,A,627


From previous code we can see that we cannot make a split which divides data in more than 4 splits, so we will choose to split data in 4 splits.

In [12]:
splits_num = [[0 for i in range(384)] for j in range(4)]
splits = [[],[],[],[]]

In [13]:
from tqdm import tqdm

classes_sorted = (labels.label.value_counts().index.values.tolist())
classes_sorted.reverse()

for i in tqdm(classes_sorted):
    clusters = labels[labels.label==i].clusters.value_counts().index.values.tolist()
    new_clusters = []
    for cluster in clusters:
        if (cluster in splits[0]) or (cluster in splits[1]) or (cluster in splits[2]) or (cluster in splits[3]):
            pass
        else:
            new_clusters.append(cluster)
    for cluster in new_clusters:
        splits = [x for x,_ in sorted(zip(splits,splits_num), key = lambda l: l[1][i], reverse=True)]
        splits_num = sorted(splits_num, key= lambda l: l[i], reverse=True)
        splits[-1].extend([cluster])
        indices = labels[labels.clusters==cluster].label.value_counts().index.values.tolist()
        counts = labels[labels.clusters==cluster].label.value_counts().tolist()
        for j in range(len(indices)):
            splits_num[-1][indices[j]]+=counts[j]

100%|██████████| 384/384 [00:07<00:00, 54.48it/s]


In [21]:
labels[labels.clusters.isin(splits[3])].label.value_counts()

357    377
178    262
23     247
215    174
351    134
      ... 
335      3
291      2
196      2
72       2
206      1
Name: label, Length: 384, dtype: int64

And final step is to save data splits.

In [22]:
os.mkdir("./data/enzyme_data/splits")

for i in range(4):
    labels[labels.clusters.isin(splits[i])][['chain_id','label']].to_csv("./data/enzyme_data/splits/split" + str(i) + ".txt",header=False,index=False)

### Protein Domain Motion Dataset

Firstly, we need to prepare cluster fatching object.

In [2]:
clusters = RcsbPdbClusters(cluster_dir = ".\data\domain_data")

2023-04-30 18:22:57.600 | INFO     | libraries.pdb:_fetch_cluster_file:80 - cluster file path: .\data\domain_data\pdb_clusters_30.txt


Now it is important to load the data and explore it.

In [3]:
df = pd.read_csv('./data/domain_data/structural_rearrangement_data.csv')
df.head()

,Unnamed: 0,level_0,index,PSCID,Protein Name,Free form,Bound form,Ligands,Classification(?),motion_type,Free PDB,Free Chains,Bound PDB,Bound Chains
0,0,0,0,CD.1,HYPOTHETICAL OXIDOREDUCTASE YIAK,1nxu_AB,1s20_AB,"2xNAD,2xTLA",200004,coupled_domain_motion,1nxu,AB,1s20,AB
1,1,1,1,CD.2,ADENYLATE KINASE,4ake_A,2eck_A,"ADP,AMP",200003,coupled_domain_motion,4ake,A,2eck,A
2,2,2,2,CD.3,GLUCOKINASE,1q18_AB,1sz2_AB,2xBGC,200003,coupled_domain_motion,1q18,AB,1sz2,AB
3,3,3,3,CD.4,LACTOFERRIN,1lfh_A,1lfi_A,"2xCU,2xNAG",110103,coupled_domain_motion,1lfh,A,1lfi,A
4,4,4,4,CD.5,ELONGATION FACTOR 2,1n0v_D,1n0u_A,SO1,110002,coupled_domain_motion,1n0v,D,1n0u,A


We can see that we have a table in which we have numerous columns. But the most interesting for us are: classes (motion_type) and inputs (Free PDB and Free Chains). The reason for choosing free proteins instead of bound ones is that our task is to predict types of motions protein can have from minimum potential well energy and those configurations are represented in free forms.

In [4]:
df['classes'] = df['motion_type'].replace({"coupled_domain_motion": 0, "independent_domain_motion":1, "independent_local_motion":2, "burying_ligand_motion":3 , 'coupled_local_motion':4, "no_significant_motion":5, "other_motion":6})
important_df = df[["Free PDB", "Free Chains", "classes"]]

To make our task easier, we will extract only necessary columns with encodings of the classes as presented in the code above. As this task includes multi chain proteins, for which data is only available per chain, we are forced to work only on single chain proteins. This is because the problem of obtaining the full multi chain structure with correct coordiantes is hard.

In [5]:
important_df = important_df.loc[important_df["Free Chains"].str.len() == 1]
important_df.head()

,Free PDB,Free Chains,classes
1,4ake,A,0
3,1lfh,A,0
4,1n0v,D,0
6,2p0m,A,0
7,3cze,A,0


Now we need to make sure that each chain appears only under one class. This task can be extended to multilabel problem, but for the purpose of this porject we are going to stick only to chains which belong to a single class.

In [6]:
important_df = important_df.drop(important_df[important_df.classes==6].index)
important_df = important_df.drop_duplicates(subset=["Free Chains", "Free PDB"], keep = False)

Now the question is raised to why we deleted the whole class 6. This class corresponds to other motion. It turns out that all elements of class 3 experience other motion, but with different ligands, so we decided to leave them out. On the other hand, for all other duplicates, we are deleting all repetances. Now we have obtained sequences which can be used for training, but important thing is to firstly check whether they are present in our LMDB database of structures.

In [7]:
database = LMDBDataset("./data/pdb_data")

warnings.filterwarnings("error")
flex = "msqf"

important_df["chain_id"] = important_df["Free PDB"] + "." + important_df["Free Chains"]

missing_ids = []
for id in tqdm(set(important_df.chain_id)):
    PDBid, chain = id.split(".")
    if(PDBid in database):
        structure = database[PDBid]["structure"]
        x = DataPreProcessorForGNM(type_flexibility = flex, num_modes=10)
        protein_chain = structure[(structure.chain_id == chain) & struc.filter_amino_acids(structure)] 
        if(not is_chain_valid(protein_chain)):
            missing_ids.extend([id])
    else:
        missing_ids.extend([id])
len(missing_ids)

100%|██████████| 497/497 [00:02<00:00, 200.31it/s]


256

Now we are only going to take sequences for which we have obtained full structures in our LMDB database.

In [8]:
important_df = important_df[~important_df.chain_id.isin(missing_ids)]
important_df

,Free PDB,Free Chains,classes,chain_id
1,4ake,A,0,4ake.A
6,2p0m,A,0,2p0m.A
7,3cze,A,0,3cze.A
9,2ex0,B,0,2ex0.B
14,2gg4,A,0,2gg4.A
...,...,...,...,...
770,2r60,A,5,2r60.A
783,1hf8,A,5,1hf8.A
784,3c7m,B,5,3c7m.B
785,2q22,C,5,2q22.C


Now that we have done this, it is important to attach clusters to sequences.

In [9]:
clustered = []
for line in important_df.values:
    clustered.append([line[0],line[1],line[2], clusters.get_seqclust(pdb_code=line[0],chain_id=line[1])])
clustered_df = pd.DataFrame(clustered, columns = ['Free PDB', 'Free Chains', 'classes', 'clusters'])
clustered_df

,Free PDB,Free Chains,classes,clusters
0,4ake,A,0,82
1,2p0m,A,0,2915
2,3cze,A,0,4761
3,2ex0,B,0,6623
4,2gg4,A,0,188
...,...,...,...,...
236,2r60,A,5,43904
237,1hf8,A,5,14827
238,3c7m,B,5,26867
239,2q22,C,5,102879


Now that we have achieved this, it is important to check number of sequences per cluster.

In [10]:
clustered_df.clusters.value_counts()[clustered_df.clusters.value_counts()>1]

316     2
370     2
1860    2
Name: clusters, dtype: int64

We can see that there are only 3 clusters with more sequences than 1. Now let us check number of sequences per classes.

In [11]:
clustered_df.classes.value_counts()

5    95
2    46
4    42
3    21
1    19
0    18
Name: classes, dtype: int64

Now we will apply brute force splitting to make 5 reasonable splits.

In [12]:
test_split = [316, 370, 1860]
clustered_df[clustered_df.clusters.isin(test_split)]

,Free PDB,Free Chains,classes,clusters
40,1pa1,A,4,316
50,2iyt,A,4,370
56,1zuh,A,4,370
83,2qct,B,2,316
134,2fyo,A,3,1860
170,1t7n,A,5,1860


In [13]:
import numpy as np
class_count = [18, 19, 46, 21, 42, 95]
splits_num = [[0, 0, 0, 0, 2, 0],[0, 0, 1, 0, 1, 0],[0, 0, 0, 1, 0, 1],[0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0]]
splits = [[370],[316], [1860], [], []]

In [14]:
df = clustered_df[~clustered_df.clusters.isin(test_split)]

for i in range(6):
    class_per_split = class_count[i]//5
    pos = 0
    for j in range(5):
        if j!=4:
            splits[j].extend(df[df.classes==i].clusters.iloc[pos: pos+class_per_split-splits_num[j][i]].tolist())
            splits_num[j][i]=class_per_split
            pos += class_per_split-splits_num[j][i]
        else:
            splits[j].extend(df[df.classes==i].clusters.iloc[pos:pos+class_count[i]-class_per_split*4-splits_num[j][i]].tolist())
            splits_num[j][i]= class_count[i]-class_per_split*4
print(splits)

[[370, 82, 2915, 4761, 33319, 3226, 7654, 140, 3473, 5053, 18795, 822, 39, 2025, 9601, 1155, 335, 21873, 55, 5248, 16648, 33020, 2675, 5435, 3900, 6935, 1, 18876, 8079, 19014, 43215, 33342, 11190, 2219, 60889, 14811, 32967, 155, 4107, 43440, 33071, 43439, 16548, 18798, 60225], [316, 82, 2915, 4761, 33319, 3226, 7654, 140, 3473, 5053, 18795, 822, 39, 2025, 9601, 335, 21873, 55, 5248, 16648, 33020, 2675, 5435, 3900, 6935, 21806, 1, 18876, 8079, 19014, 43215, 33342, 11190, 2219, 60889, 14811, 32967, 155, 4107, 43440, 33071, 43439, 16548, 18798, 60225], [1860, 82, 2915, 4761, 33319, 3226, 7654, 140, 3473, 5053, 18795, 822, 39, 2025, 9601, 1155, 335, 21873, 55, 16648, 33020, 2675, 5435, 3900, 6935, 21806, 61479, 1, 18876, 8079, 19014, 43215, 33342, 11190, 2219, 60889, 14811, 32967, 155, 4107, 43440, 33071, 43439, 16548, 18798], [82, 2915, 4761, 33319, 3226, 7654, 140, 3473, 5053, 18795, 822, 39, 2025, 9601, 1155, 335, 21873, 55, 5248, 16648, 33020, 2675, 5435, 3900, 6935, 21806, 61479, 1, 1

In [15]:
clustered_df["chain"] = clustered_df["Free PDB"] + "." + clustered_df["Free Chains"]

And finally when we split the data, we need to store it for future use.

In [18]:
clustered_df[["chain","classes"]].to_csv("./data/domain_data/chain_functions.txt",header=False,index=False)

In [21]:
os.mkdir("./data/domain_data/splits")

for i in range(5):
    clustered_df[clustered_df.clusters.isin(splits[i])][["chain","classes"]].to_csv("./data/domain_data/splits/split"+ str(i) + ".txt",header=False,index=False)